In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# imports
import numpy as np
import tensorflow as tf

from tensorflow.contrib import learn
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib

tf.logging.set_verbosity(tf.logging.INFO)

In [ ]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN"""
    
    #input layer
    input_layer = tf.reshape(features, [-1, 128, 128, 1])
    
    #Convolutional Layer #1 and Pooling Layer #1
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=24,
        kernel_size=[6,6],
        padding="same",
        activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(inputs=cov1, pool_size=[2,2], strides=2)
    
    #Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=32,
        kernel_size=[3,3],
        padding="same",
        activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=cov2, pool_size=[2,2], strides=2)

    #Convolutional Layer #3 and Pooling Layer #3
    conv3 = tf.layers.conv2d(
        inputs=pool2,
        filters=48,
        kernel_size=[3,3],
        padding="same",
        activation=tf.nn.relu)
    pool3 = tf.layers.max_pooling2d(inputs=cov3, pool_size=[2,2], strides=2)

    #Convolutional Layer #4 and Pooling Layer #4
    conv4 = tf.layers.conv2d(
        inputs=pool3,
        filters=64,
        kernel_size=[3,3],
        padding="same",
        activation=tf.nn.relu)
    pool4 = tf.layers.max_pooling2d(inputs=cov4, pool_size=[2,2], strides=2)

    #Convolutional Layer #5 and Pooling Layer #5
    conv5 = tf.layers.conv2d(
        inputs=pool4,
        filters=128,
        kernel_size=[3,3],
        padding="same",
        activation=tf.nn.relu)
    pool5 = tf.layers.max_pooling2d(inputs=cov5, pool_size=[2,2], strides=2)

    #Convolutional Layer #6 and Pooling Layer #6
    conv6 = tf.layers.conv2d(
        inputs=pool5,
        filters=256,
        kernel_size=[3,3],
        padding="same",
        activation=tf.nn.relu)
    pool6 = tf.layers.max_pooling2d(inputs=cov6, pool_size=[2,2], strides=2)
    
    #Dense Layer #1
    pool6_flat = tf.reshape(pool6, [-1, 2*2*256])
    dense1 = tf.layers.dense(inputs=pool6_flat, units=1024, activation=tf.nn.relu)
    '''
    dropout = tf.layers.dropout(
        inputs=dense1, rate=0.5, training=mode==learn.ModeKeys.TRAIN)
    '''
    
    #Dense Layer #2
    dense2= tf.layers.dense(inputs=dense1, units=512, activation=tf.nn.relu)
    '''
    dropout = tf.layers.dropout(
        inputs=dense1, rate=0.5, training=mode==learn.ModeKeys.TRAIN)
    '''
    
    #Logits Layer
    logits = tf.layers.dense(inpout=dense2, units=1) #without linear activation, might be wrong....
    
    loss = None
    train_op = None
    
    #Calculate Loss (for both train and evaluation)
    #if mode != learn.ModeKeys.INFER
    loss = tf.train.AdamOptimizer()

In [2]:
import numpy as np
import scipy
import keras

ImportError: No module named keras